In [2]:
import pandas as pd
import os
from tqdm import tqdm
from IPython.display import clear_output    


In [3]:
def list_files_in_directory(directory_path, file_extenion = None):
    try:
        all_files = os.listdir(directory_path)
        
        if file_extenion:
            files = [f for f in all_files if f.endswith(file_extenion)]
            
        else:
            files = all_files
            
        return files
            
    except FileNotFoundError:
        print(f"Error: The directory {directory_path} does not exist")
        return []
    except Exception as e:
        print(f"An error occured: {e}")
        return []

In [4]:
rwf_train_non_violence_video_path =  "video_data/RWF-2000/train/NonFight"
rwf_train_violence_video_path = "video_data/RWF-2000/train/Fight"
rwf_val_non_violence_video_path = "video_data/RWF-2000/val/NonFight"
rwf_val_violence_video_path = "video_data/RWF-2000/val/Fight"

In [5]:
rwf_train_non_violence_video_files = list_files_in_directory(rwf_train_non_violence_video_path)
rwf_train_violence_video_files = list_files_in_directory(rwf_train_violence_video_path)
rwf_val_non_violence_video_files = list_files_in_directory(rwf_val_non_violence_video_path)
rwf_val_violence_video_files = list_files_in_directory(rwf_val_violence_video_path)

In [6]:
print(len(rwf_val_violence_video_files),
len(rwf_train_violence_video_files),
len(rwf_val_non_violence_video_files),
len(rwf_train_non_violence_video_files))

200 789 200 800


In [8]:
rwf_val_non_violence_video_files = rwf_val_non_violence_video_files[104:]

In [7]:

from backend.utils.feature_extraction import ViolenceFeatureExtractor
from backend.utils.data_preprocessing import preprocess_data

# Absolute paths to your models
detection_model_path = "backend/models/yolo8n.pt"
pose_model_path = "backend/models/yolo8n.pt"

# Check if the model files exist
if not os.path.exists(detection_model_path):
    raise FileNotFoundError(f"Detection model not found at {detection_model_path}")

if not os.path.exists(pose_model_path):
    raise FileNotFoundError(f"Pose model not found at {pose_model_path}")

# Initialize the feature extractor
extractor = ViolenceFeatureExtractor()

# Path to the final CSV where data will be appended
final_csv_path = 'extracted_feature_data/val_non_violence_data.csv'

# Path to save the output CSV
output_csv_path = 'extracted_feature_data/sample.csv'

full_log = []

# tqdm loop with description
for video_name in tqdm(rwf_val_non_violence_video_files, desc="Processing Videos"):
    try:
        extractor.reset()
        video_path = os.path.join(rwf_val_non_violence_video_path, video_name)

        print(f"\n📹 Processing video: {video_name}")
        frame_width, frame_height, interaction_count = extractor.process_video(
            video_path,
            output_csv_path,
            output_folder=None,
            show_video=False,
            save_video=False
        )
        clear_output(wait=True)

        df_csv = pd.read_csv(output_csv_path)
        df_preprocessed, expected_columns = preprocess_data(df_csv, frame_width, frame_height)

        # Skip appending if there are no interactions (empty dataframe)
        if df_preprocessed.empty:
            log_entry = f"❌ {video_name}: No interactions found"
            print(log_entry)
            full_log.append(log_entry)
            continue  # Skip to the next video

        # Check if the final CSV exists and has data
        if os.path.exists(final_csv_path) and os.path.getsize(final_csv_path) > 0:
            # Append to the existing CSV
            df_preprocessed.to_csv(final_csv_path, mode='a', header=False, index=False)
        else:
            # Create a new CSV with the appropriate header
            df_preprocessed.to_csv(final_csv_path, mode='w', header=True, index=False)

        log_entry = f"✅ {video_name}: {len(df_preprocessed)} interactions saved"
        print(log_entry)

    except Exception as e:
        log_entry = f"❌ Error processing {video_name}: {e}"
        print(log_entry)

    full_log.append(log_entry)



Processing Videos: 100%|██████████| 96/96 [10:26<00:00,  6.52s/it]

✅ ZwhC7Kyg_0.avi: 485 interactions saved


In [8]:

from backend.utils.feature_extraction import ViolenceFeatureExtractor
from backend.utils.data_preprocessing import preprocess_data

# Absolute paths to your models
detection_model_path = "backend/models/yolo8n.pt"
pose_model_path = "backend/models/yolo8n.pt"

# Check if the model files exist
if not os.path.exists(detection_model_path):
    raise FileNotFoundError(f"Detection model not found at {detection_model_path}")

if not os.path.exists(pose_model_path):
    raise FileNotFoundError(f"Pose model not found at {pose_model_path}")

# Initialize the feature extractor
extractor = ViolenceFeatureExtractor()

# Path to the final CSV where data will be appended
final_csv_path = 'extracted_feature_data/val_violence_data.csv'

# Path to save the output CSV
output_csv_path = 'extracted_feature_data/sample.csv'

full_log = []

# tqdm loop with description
for video_name in tqdm(rwf_val_violence_video_files, desc="Processing Videos"):
    try:
        extractor.reset()
        video_path = os.path.join(rwf_val_violence_video_path, video_name)

        print(f"\n📹 Processing video: {video_name}")
        frame_width, frame_height, interaction_count = extractor.process_video(
            video_path,
            output_csv_path,
            output_folder=None,
            show_video=False,
            save_video=False
        )
        clear_output(wait=True)

        df_csv = pd.read_csv(output_csv_path)
        df_preprocessed, expected_columns = preprocess_data(df_csv, frame_width, frame_height)

        # Skip appending if there are no interactions (empty dataframe)
        if df_preprocessed.empty:
            log_entry = f"❌ {video_name}: No interactions found"
            print(log_entry)
            full_log.append(log_entry)
            continue  # Skip to the next video

        # Check if the final CSV exists and has data
        if os.path.exists(final_csv_path) and os.path.getsize(final_csv_path) > 0:
            # Append to the existing CSV
            df_preprocessed.to_csv(final_csv_path, mode='a', header=False, index=False)
        else:
            # Create a new CSV with the appropriate header
            df_preprocessed.to_csv(final_csv_path, mode='w', header=True, index=False)

        log_entry = f"✅ {video_name}: {len(df_preprocessed)} interactions saved"
        print(log_entry)

    except Exception as e:
        log_entry = f"❌ Error processing {video_name}: {e}"
        print(log_entry)

    full_log.append(log_entry)



Processing Videos: 100%|██████████| 200/200 [22:40<00:00,  6.80s/it]

✅ zOqs7Oh9oDM_3.avi: 14 interactions saved


In [7]:
rwf_train_violence_video_files = rwf_train_violence_video_files[36:]

In [ ]:

from backend.utils.feature_extraction import ViolenceFeatureExtractor
from backend.utils.data_preprocessing import preprocess_data

# Absolute paths to your models
detection_model_path = "backend/models/yolo8n.pt"
pose_model_path = "backend/models/yolo8n.pt"

# Check if the model files exist
if not os.path.exists(detection_model_path):
    raise FileNotFoundError(f"Detection model not found at {detection_model_path}")

if not os.path.exists(pose_model_path):
    raise FileNotFoundError(f"Pose model not found at {pose_model_path}")

# Initialize the feature extractor
extractor = ViolenceFeatureExtractor()

# Path to the final CSV where data will be appended
final_csv_path = 'extracted_feature_data/train_violence_data.csv'

# Path to save the output CSV
output_csv_path = 'extracted_feature_data/sample.csv'

full_log = []

# tqdm loop with description
for video_name in tqdm(rwf_train_violence_video_files, desc="Processing Videos"):
    try:
        extractor.reset()
        video_path = os.path.join(rwf_train_violence_video_path, video_name)

        print(f"\n📹 Processing video: {video_name}")
        frame_width, frame_height, interaction_count = extractor.process_video(
            video_path,
            output_csv_path,
            output_folder=None,
            show_video=False,
            save_video=False
        )
        clear_output(wait=True)

        df_csv = pd.read_csv(output_csv_path)
        df_preprocessed, expected_columns = preprocess_data(df_csv, frame_width, frame_height)

        # Skip appending if there are no interactions (empty dataframe)
        if df_preprocessed.empty:
            log_entry = f"❌ {video_name}: No interactions found"
            print(log_entry)
            full_log.append(log_entry)
            continue  # Skip to the next video

        # Check if the final CSV exists and has data
        if os.path.exists(final_csv_path) and os.path.getsize(final_csv_path) > 0:
            # Append to the existing CSV
            df_preprocessed.to_csv(final_csv_path, mode='a', header=False, index=False)
        else:
            # Create a new CSV with the appropriate header
            df_preprocessed.to_csv(final_csv_path, mode='w', header=True, index=False)

        log_entry = f"✅ {video_name}: {len(df_preprocessed)} interactions saved"
        print(log_entry)

    except Exception as e:
        log_entry = f"❌ Error processing {video_name}: {e}"
        print(log_entry)

    full_log.append(log_entry)



Processing Videos:   2%|▏         | 16/753 [01:42<1:08:56,  5.61s/it]

✅ 3kpviz7lAMY_0.avi: 4 interactions saved

📹 Processing video: 3kpviz7lAMY_1.avi
Input resolution: 428x240
Frame skip: 1
Batch size: 1
Processing video: 428x240 at 30.0 fps
Using frame_skip: 1, batch_size: 1
Processing frame 0/150 (0.0%)Original size: 428x240, Resized size: 640x358
Padding: (pad_w: 0, pad_h: 141)
Processing frame 1/150 (0.7%)Original size: 428x240, Resized size: 640x358
Padding: (pad_w: 0, pad_h: 141)
Processing frame 2/150 (1.3%)Original size: 428x240, Resized size: 640x358
Padding: (pad_w: 0, pad_h: 141)
Processing frame 3/150 (2.0%)Original size: 428x240, Resized size: 640x358
Padding: (pad_w: 0, pad_h: 141)
Processing frame 4/150 (2.7%)Original size: 428x240, Resized size: 640x358
Padding: (pad_w: 0, pad_h: 141)
Processing frame 5/150 (3.3%)Original size: 428x240, Resized size: 640x358
Padding: (pad_w: 0, pad_h: 141)
Processing frame 6/150 (4.0%)Original size: 428x240, Resized size: 640x358
Padding: (pad_w: 0, pad_h: 141)
Processing frame 7/150 (4.7%)Original size:

In [2]:
from video_data.video_data_downloader import download_dataset

download_dpath = download_dataset()

TypeError: dataset_download() got an unexpected keyword argument 'progress'